In [ ]:
import sys, pickle, os, json, re, time, random, logging, pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, scipy, sklearn, networkx as nx, importlib; sys.path.append('./Dropbox'); import tools; importlib.reload(tools)
import pandas as pd
merged_df=pd.read_excel("df_download_urls_intro.xlsx")
merged_df = merged_df.drop_duplicates(subset=["content"], keep=False)

def prompt_maker(text,cve):
    prompt_message = [{"role": "user", "content": "Here's page's content: \"" + text+"\"Now,the page is end. You task is :Write out the original text of the \"longest continuous text\" of the description or summary of+"+cve+ "in the page content. You should directly copy the text from the page content without any modification or any other word like \"the longest continuous text is\". If the page disscuss multiple CVEs or target CVE and in a long text without clear \"description\" or \"summary\" section, you can return the first section that dissuss the CVE. If the page does not include any word about the CVE, you return \"this page does not include any information about %CVE_name%\""}]
    return prompt_message

alltext = merged_df["content"].tolist()
alltext=[tools.tokencut(i,4096*2) for i in alltext]
allcve = merged_df["cve"].tolist()
code_name="cve_description"
prompts = [prompt_maker(alltext[i],allcve[i]) for i in range(len(alltext))]
print(len(str(prompts)))
jsonl_file = tools.create_jsonl(prompts,model='gpt4', temp=0.7, token=4*4096, jsonlname=code_name, possible_output=300)
ids = tools.upload_RUN_PAY_jsonl(jsonl_file, code_name)

In [ ]:
ans=tools.auto_down_ans(ids['batch_id']))
merged_df['short_description'] = ans
merged_df['source']='cvenvdplus'

In [ ]:
rq3df = merged_df.copy()
rq3df['raw_order'] = 999

cve_counter = {}

for idx, row in rq3df.iterrows():
    if row['source'] != 'mostpop2024':
        continue
    cve_value = row['cve']
    if cve_value not in cve_counter:
        rq3df.at[idx, 'raw_order'] = 0
        cve_counter[cve_value] = 0
    else:
        if cve_counter[cve_value] < 9:
            cve_counter[cve_value] += 1
            rq3df.at[idx, 'raw_order'] = cve_counter[cve_value]
            
rq3df_sorted = rq3df.sort_values(by=['source', 'cve', 'raw_order'], na_position='last').reset_index(drop=True)
rq3df_sorted = rq3df_sorted.drop_duplicates(subset=['short_description'], keep='first')

In [ ]:
cve_df_dict = {cve: df for cve, df in rq3df_sorted.groupby('cve')}
#get first key's value
cves_key=[]
for i in cve_df_dict.keys():
    cves_key.append(i)
total_filtered_df_list=[]
for index in range(len(cves_key)):
    subdf=cve_df_dict[cves_key[index]]
    #reset subdf index
    subdf.reset_index(drop=True, inplace=True)
    subdf = subdf.head(50)
    short_description_dict = {
        str(i): desc[:1024] for i, desc in enumerate(subdf['short_description'])
    }

    def prompt_maker(text):
        prompt_message = [
            {
                "role": "user",
                "content": (
                    "I will provide you with a dictionary where the keys are article IDs and the values are short descriptions of the articles. "
                    "Here is my input dictionary:\n"+str(text)+"\nNow, the my end is end, your task is: "
                    "Each value in the dictionary is a brief description of an article. Some of these descriptions are 'almost identical with <5 words difference,' or 'as the same as' each other, "
                    "Rule 1: I would like to filter out these nearly identical descriptions or the same descriptions. The rule is for the descriptions that are 'almost identical' or 'as the same as' each other, only add the first id of the descriptions to your output list. For the unique descriptions, add them to your output list in the order they appear in the input dictionary. "
                    "Rule 2: top once the total length of the list reaches 10. "
                    "For example, if id 1 and id 2 have the same description, only add id 1 to the output list. If id 3 and id 4 have different descriptions and don't have any descriptions that are 'almost identical (<5 words difference)' or 'as the same as' each other, add both id 3 and id 4 to the output list. The final output ids should refer to a group of short article decsriptions that are unique, and DON'T have any descriptions that are 'almost identical' or 'as the same as' each other. for any other descriptions in this group. "
                    "Rule 3: if the article description is not English, you can ignore it, even it is unique, you still should not add it to the output list. "
                    "Your response has two parts: 1 A list that contain unique descriptions (only first 50 words) and their ids,2 A json that only summarize the ids of the unique descriptions. "
                    "A example of the output is \" Part 1:[[%first_50_words_of_description1%,%corresponding_id1%],[%first_50_words_of_description2%,%corresponding_id2%]], Part 2: json```{\"article_ids\": [1, 2]}``` Then stop output as soon as possible, without any other words"
                )
            }
        ]
        return prompt_message

    prompt = prompt_maker(str(short_description_dict))
    prompts = [prompt]
    ans = tools.ask_group_link(prompts, token=4*4096, temp=0.7, model="gpt4", streamprint=False, max_workers=8, weight='auto',forcegpt=True)
    import json
    listof_select_index=tools.ans_last_code_as_json(ans)
    #only the list of selected index on subdf
    listof_select_index_int=[int(i) for i in listof_select_index['article_ids']]
    subdf_filtered = subdf.loc[listof_select_index_int]
    total_filtered_df_list.append(subdf_filtered)
    print('Finsihed:',index)


In [ ]:
unique_content = []
for singledf in total_filtered_df_list:
    unique_content.extend(singledf['short_description'].tolist())
len(unique_content)

rq3df_sorted['de-duplicate_order'] = 999
cve_counter={}
for idx, row in rq3df_sorted.iterrows():
    if row['short_description'] in unique_content:
        cve_value = row['cve']
        if cve_value not in cve_counter:
            rq3df_sorted.at[idx, 'de-duplicate_order'] = 0
            cve_counter[cve_value] = 0
        else:
            if cve_counter[cve_value] < 9:
                cve_counter[cve_value] += 1
                rq3df_sorted.at[idx, 'de-duplicate_order'] = cve_counter[cve_value]
        #remove the content from unique_content
        unique_content.remove(row['short_description'])
                
rq3df_sorted.to_excel('intro.xlsx', index=False)

In [ ]:
rq3df_sorted_toprocess = rq3df_sorted[(rq3df_sorted['de-duplicate_order'] != 999) | (rq3df_sorted['raw_order'] != 999)]
rq3df_sorted_toprocess.to_excel('intro.xlsx', index=False)